# 1) Preparación previa

### Carga de librerías

In [ ]:
import pandas as pd
import numpy as np
import re

### Lectura del dataset original de Properati

In [ ]:
data = pd.read_csv("https://media.githubusercontent.com/media/Agustin-Bulzomi/Projects/main/Programming/Digital%20House%20(Python)/Support%20Files/Project%201/Properati.csv", index_col=0)
data.head(5)

### Análisis de nulos según columnas

In [ ]:
nulos = data.isnull().sum()

In [ ]:
nulos_porcentaje = nulos / data.shape[0] * 100
nulos_porcentaje

En base a los resultados se llegó a las siguientes conclusiones:

1) Se tomará la superficie cubierta en vez de la total. En el anterior desafío se creó una función para tomar una mezcla de ambas ignorando las inconsistencias pero daba más nulos aún.

2) Se tomará el precio aprox en dólares en vez del precio per m2 en dólares. Al mismo se dividirá por la superficie cubierta para tener el valor por m2

3) "rooms" tiene pocos valores, se tendrá que imputar los datos faltantes

4) "description" y "title" servirán para obtener información extra

### Separación de columna con muchas ubicaciones

In [ ]:
# La columna "place_with_parent_names" tiene información separada con '|'. Se separa para obtener info adicional
separar_zona = data["place_with_parent_names"].str.split('|', expand = True)
separar_zona.columns = ['??', 'Pais', 'Zona', 'Partido', 'Barrios', 'Country', 'Otra']

### Agregado de la nueva información en nuevas columnas

In [ ]:
data_concat = pd.concat([data, separar_zona], axis=1)
data_concat.head(10)

### Creación de la columna de precios por metros cuadrados usando la superficie cubierta

In [ ]:
data_concat['precio_usd_por_m2'] = data_concat.price_aprox_usd/data_concat.surface_covered_in_m2 

# 2) Imputación

Debido a la insuficiente cantidad de datos de ambientes, vamos a intentar obtener más

### 1) Imputación en base a título y descripción

Descripción

In [ ]:
patron_amb = "(?P<ambiente>\d\s)((A|a)(M|m)(B|b))"
regex_amb = re.compile(patron_amb)

data_amb_serie = data_concat["description"]
data_amb_match = data_amb_serie.apply(lambda x: x if x is np.NaN else regex_amb.search(x))

mask_amb_notnull = data_amb_match.notnull()

data_ambientes = data_amb_match[mask_amb_notnull].apply(lambda x: x.group("ambiente"))

data_concat.loc[mask_amb_notnull, 'ambientes_desc'] = \
    data_amb_match[mask_amb_notnull].apply(lambda x: x.group('ambiente'))

In [ ]:
data_concat.loc[mask_amb_notnull, ["description", "ambientes_desc"]]

Título

In [ ]:
patron_amb2 = "(?P<ambiente_title>\d\s)((A|a)(M|m)(B|b))"
regex_amb2 = re.compile(patron_amb2)

data_amb_serie2 = data_concat["title"]
data_amb_match2 = data_amb_serie2.apply(lambda x: x if x is np.NaN else regex_amb2.search(x))

mask_amb_notnull2 = data_amb_match2.notnull()

data_ambientes2 = data_amb_match2[mask_amb_notnull2].apply(lambda x: x.group("ambiente_title"))

data_concat.loc[mask_amb_notnull2, 'ambientes_t'] = \
    data_amb_match2[mask_amb_notnull2].apply(lambda x: x.group('ambiente_title'))

In [ ]:
data_concat.loc[mask_amb_notnull2, ["ambientes_desc", "ambientes_t"]]

#### Unificación de la nueva información

Se crea una función para resumir ambas columnas en una nueva

In [ ]:
def limpieza_amb(ambientes_desc, ambientes_t):
    if pd.isnull(ambientes_desc) and pd.isnull(ambientes_t):
        ambientes = np.NaN
    elif pd.isnull(ambientes_desc):
        ambientes = ambientes_t
    else:
        ambientes = ambientes_desc
    return ambientes

In [ ]:
# Se aplica la función
data_concat["ambientes"] = data_concat.apply(lambda data_concat: limpieza_amb(data_concat['ambientes_desc'],data_concat['ambientes_t']),axis=1)
data_concat.head(15)

In [ ]:
data_concat.shape

In [ ]:
data_concat.ambientes.notnull().sum() / data_concat.shape[0] * 100

Se  crea una función para resumir la información entre la nueva columna y rooms.

En la enorme mayoría de los casos en donde se tenía el dato de rooms original, la cantidad de ambientes obtenida por imputación concordaba con el valor de rooms original. Esto indica que ambos términos son intercambiables al menos en este dataset.

In [ ]:
def limpieza_amb2(rooms, ambientes):
    if pd.isnull(rooms) and pd.isnull(ambientes):
        ambientes_train = 0
    elif pd.isnull(rooms):
        ambientes_train = ambientes
    else:
        ambientes_train = int(rooms)
    return int(ambientes_train)

In [ ]:
# Se aplica la función. Se llama a la nueva variable "train" pues es la que será usada para entrenar al modelo
data_concat["ambientes_train"] = data_concat.apply(lambda x: limpieza_amb2(x['rooms'],x['ambientes']),axis=1)
data_concat.ambientes_train.value_counts()

### 2) Imputación en base a la superficie

Tomando la mediana de las superficies agrupadas según ambientes definir un punto medio entre cada mediana.

La misma nos permitiría definir un divisor que delimite cuándo una superficie es más probable que pertenezca a una cantidad de ambientes. Al ser una imputación no tan certera, se dejará afuera de la serie "train".

In [ ]:
# Se calcula cuánta información nueva podría obtenerse
superficie_not_null = data_concat['surface_covered_in_m2'].notnull()
ambientes_zero = data_concat['ambientes_train'] == 0
filtro = superficie_not_null & ambientes_zero
print(filtro.sum())

### Divisores de ambientes

In [ ]:
amb_1 = data_concat['ambientes_train'] == 1
amb_2 = data_concat['ambientes_train'] == 2
amb_3 = data_concat['ambientes_train'] == 3
amb_4 = data_concat['ambientes_train'] == 4
amb_5 = data_concat['ambientes_train'] == 5
amb_6 = data_concat['ambientes_train'] == 6
amb_7 = data_concat['ambientes_train'] == 7

divisor1 = (data_concat[amb_1].surface_covered_in_m2.median() + data_concat[amb_2].surface_covered_in_m2.median())/2
divisor2 = (data_concat[amb_2].surface_covered_in_m2.median() + data_concat[amb_3].surface_covered_in_m2.median())/2
divisor3 = (data_concat[amb_3].surface_covered_in_m2.median() + data_concat[amb_4].surface_covered_in_m2.median())/2
divisor4 = (data_concat[amb_4].surface_covered_in_m2.median() + data_concat[amb_5].surface_covered_in_m2.median())/2
divisor5 = (data_concat[amb_5].surface_covered_in_m2.median() + data_concat[amb_6].surface_covered_in_m2.median())/2
divisor6 = (data_concat[amb_6].surface_covered_in_m2.median() + data_concat[amb_7].surface_covered_in_m2.median())/2
divisor7 = (data_concat[amb_7].surface_covered_in_m2.median() + data_concat[amb_7].surface_covered_in_m2.max())/2

# Como no hay de 8 ambientes, se utiliza el valor máximo de 7 ambientes como tope para calcular el divisor 7

In [ ]:
# Se crea una función para asignar ambientes según los divisores

def asignar_ambientes_segun_superficie(surface_covered_in_m2):
    #if superficie.isnull():
    #    return 0
    #elif 0 < superficie <= divisor1:
    if 0 < surface_covered_in_m2 <= divisor1:
        return 1
    elif divisor1 < surface_covered_in_m2 <= divisor2:
        return 2
    elif divisor2 < surface_covered_in_m2 <= divisor3:
        return 3
    elif divisor3 < surface_covered_in_m2 <= divisor4:
        return 4
    elif divisor4 < surface_covered_in_m2 <= divisor5:
        return 5
    elif divisor5 < surface_covered_in_m2 <= divisor6:
        return 6
    elif divisor6 < surface_covered_in_m2 <= divisor7:
        return 7
    else:
        return np.NaN

In [ ]:
# Se aplica la función para crear una columna de ambientes imputados con valores en las filas que no tienen valores de ambientes_train

data_concat["ambientes_imputados"] = data_concat.apply(lambda x: asignar_ambientes_segun_superficie(x['surface_covered_in_m2']) if int(x['ambientes_train']) == 0 else 0, axis=1)

In [ ]:
data_concat.ambientes_imputados.value_counts()

In [ ]:
# Se suman ambas columnas al ser excluyentes: ambientes_final no tiene 0, cada fila tiene un valor original o imputado

data_concat["ambientes_final"] = data_concat["ambientes_train"] + data_concat["ambientes_imputados"]

In [ ]:
data_concat

# 3) Búsqueda de amenities

Se analiza la descripción de cada fila para encontrar palabras clave que indiquen amenities con valor agregado

In [ ]:
patron_balcon = "(?P<balcon>(B|b)(A|a)(L|l)(C|c)(O|n)(N|n))"
regex_balcon = re.compile(patron_balcon)
data_balcon = data_concat["description"]
data_match_balcon = data_balcon.apply(lambda x: x if x is np.NaN else regex_balcon.search(x))
mask_notnull_balcon = data_match_balcon.notnull()
data_balcon = data_match_balcon[mask_notnull_balcon].apply(lambda x: x.group("balcon"))
data_concat.loc[mask_notnull_balcon, 'balcon'] = \
data_match_balcon[mask_notnull_balcon].apply(lambda x: x.group('balcon').lower())

patron_parrilla = "(?P<parrilla>(P|p)(A|a)(R|r)(R|r)(I|i)(L|l)(L|l)(A|a))"
regex_parrilla = re.compile(patron_parrilla)
data_parrilla = data_concat["description"]
data_match_parrilla = data_parrilla.apply(lambda x: x if x is np.NaN else regex_parrilla.search(x))
mask_notnull_parrilla = data_match_parrilla.notnull()
data_parrilla = data_match_parrilla[mask_notnull_parrilla].apply(lambda x: x.group("parrilla"))
data_concat.loc[mask_notnull_parrilla, 'parrilla'] = \
data_match_parrilla[mask_notnull_parrilla].apply(lambda x: x.group('parrilla').lower())

patron_pileta = "(?P<pileta>(P|p)(I|i)(L|l)(E|e)(T|t)(A|a))"
regex_pileta = re.compile(patron_pileta)
data_pileta = data_concat["description"]
data_match_pileta = data_pileta.apply(lambda x: x if x is np.NaN else regex_pileta.search(x))
mask_notnull_pileta = data_match_pileta.notnull()
data_pileta = data_match_pileta[mask_notnull_pileta].apply(lambda x: x.group("pileta"))
data_concat.loc[mask_notnull_pileta, 'pileta'] = \
data_match_pileta[mask_notnull_pileta].apply(lambda x: x.group('pileta').lower())

patron_patio = "(?P<patio>(P|p)(A|a)(T|t)(I|i)(O|o))"
regex_patio = re.compile(patron_patio)
data_patio = data_concat["description"]
data_match_patio = data_patio.apply(lambda x: x if x is np.NaN else regex_patio.search(x))
mask_notnull_patio = data_match_patio.notnull()
data_patio = data_match_patio[mask_notnull_patio].apply(lambda x: x.group("patio"))
data_concat.loc[mask_notnull_patio, 'patio'] = \
data_match_patio[mask_notnull_patio].apply(lambda x: x.group('patio').lower())

patron_quincho = "(?P<quincho>(Q|q)(U|u)(I|i)(N|n)(C|c)(H|h)(O|o))"
regex_quincho = re.compile(patron_quincho)
data_quincho = data_concat["description"]
data_match_quincho = data_quincho.apply(lambda x: x if x is np.NaN else regex_quincho.search(x))
mask_notnull_quincho = data_match_quincho.notnull()
data_quincho = data_match_quincho[mask_notnull_quincho].apply(lambda x: x.group("quincho"))
data_concat.loc[mask_notnull_quincho, 'quincho'] = \
data_match_quincho[mask_notnull_quincho].apply(lambda x: x.group('quincho').lower())

patron_gimnasio = "(?P<gimnasio>(G|g)(I|i)(M|m)(N|n)(A|a)(C|c|S|s)(I|i)(O|o))"
regex_gimnasio = re.compile(patron_gimnasio)
data_gimnasio = data_concat["description"]
data_match_gimnasio = data_gimnasio.apply(lambda x: x if x is np.NaN else regex_gimnasio.search(x))
mask_notnull_gimnasio = data_match_gimnasio.notnull()
data_gimnasio = data_match_gimnasio[mask_notnull_gimnasio].apply(lambda x: x.group("gimnasio"))
data_concat.loc[mask_notnull_gimnasio, 'gimnasio'] = \
data_match_gimnasio[mask_notnull_gimnasio].apply(lambda x: x.group('gimnasio').lower().replace("gimnacio", "gimnasio"))

patron_sum = "(?P<sum>(S|s)(U|u)(M|m))"
regex_sum = re.compile(patron_sum)
data_sum = data_concat["description"]
data_match_sum = data_sum.apply(lambda x: x if x is np.NaN else regex_sum.search(x))
mask_notnull_sum = data_match_sum.notnull()
data_sum = data_match_sum[mask_notnull_sum].apply(lambda x: x.group("sum"))
data_concat.loc[mask_notnull_sum, 'sala_usos_multiples'] = \
data_match_sum[mask_notnull_sum].apply(lambda x: x.group('sum').lower())

patron_cochera = "(?P<cochera>(C|c)(O|o)(C|c)(H|h)(E|e)(R|r)(A|a)|(E|e)(S|s)(T|t)(A|a)(C|c)(I|i)(O|o)(N|n)(A|a)(M|m)(I|i)(E|e)(N|n)(T|t)(O|o))"
regex_cochera = re.compile(patron_cochera)
data_cochera = data_concat["description"]
data_match_cochera = data_cochera.apply(lambda x: x if x is np.NaN else regex_cochera.search(x))
mask_notnull_cochera = data_match_cochera.notnull()
data_cochera = data_match_cochera[mask_notnull_cochera].apply(lambda x: x.group("cochera"))
data_concat.loc[mask_notnull_cochera, 'cochera'] = \
data_match_cochera[mask_notnull_cochera].apply(lambda x: x.group('cochera').lower().replace("estacionamiento", "cochera"))

patron_seguridad = "(?P<seguridad>(S|s)(E|e)(G|g)(U|u)(R|r)(I|i)(D|d)(A|a)(D|d)|(P|p)(O|o)(R|r)(T|t)(E|e)(R|r)(O|o))"
regex_seguridad = re.compile(patron_seguridad)
data_seguridad = data_concat["description"]
data_match_seguridad = data_seguridad.apply(lambda x: x if x is np.NaN else regex_seguridad.search(x))
mask_notnull_seguridad = data_match_seguridad.notnull()
data_seguridad = data_match_seguridad[mask_notnull_seguridad].apply(lambda x: x.group("seguridad"))
data_concat.loc[mask_notnull_seguridad, 'seguridad'] = \
data_match_seguridad[mask_notnull_seguridad].apply(lambda x: x.group('seguridad').lower().replace("portero", "seguridad"))

patron_jardin = "(?P<jardin>(J|j)(A|a)(R|r)(D|d)(I|i)(N|n))"
regex_jardin = re.compile(patron_jardin)
data_jardin = data_concat["description"]
data_match_jardin = data_jardin.apply(lambda x: x if x is np.NaN else regex_jardin.search(x))
mask_notnull_jardin = data_match_jardin.notnull()
data_jardin = data_match_jardin[mask_notnull_jardin].apply(lambda x: x.group("jardin"))
data_concat.loc[mask_notnull_jardin, 'jardin'] = \
data_match_jardin[mask_notnull_jardin].apply(lambda x: x.group('jardin').lower())

patron_frente = "(?P<frente>(F|f)(R|r)(E|e)(N|n)(T|t)(E|e))"
regex_frente = re.compile(patron_frente)
data_frente = data_concat["description"]
data_match_frente = data_frente.apply(lambda x: x if x is np.NaN else regex_frente.search(x))
mask_notnull_frente = data_match_frente.notnull()
data_frente = data_match_frente[mask_notnull_frente].apply(lambda x: x.group("frente"))
data_concat.loc[mask_notnull_frente, 'frente'] = \
data_match_frente[mask_notnull_frente].apply(lambda x: x.group('frente').lower())

data_concat

# 4) Eliminación de nulos, ceros, outliers e información innecesaria

## Datos innecesarios

In [ ]:
# No es de interés para el análisis actual la información inmobiliaria de tiendas

mask_not_store = data_concat['property_type'] != 'store'

In [ ]:
data_concat = data_concat[mask_not_store]

In [ ]:
data_concat.shape

## Columnas innecesarias

In [ ]:
data_sin_columnas = data_concat.drop(['operation', 'place_with_parent_names', 'place_name', 'country_name', 'state_name',
                                     'geonames_id', 'lat-lon', 'floor', 'expenses', 'properati_url', 'image_thumbnail', '??', 'price_usd_per_m2',
                                     'place_name', 'currency', 'price_aprox_local_currency', 'surface_total_in_m2', 'price_per_m2',
                                     'price_aprox_usd', "lat", "lon",  "Country", "Otra", "Barrios", "Pais", "Zona", "ambientes_desc",
                                      "ambientes_t", "ambientes", "rooms", "title", "description"], axis = 1)
data_sin_columnas.head()

## Nulos y Ceros

In [ ]:
# Se procede a eliminar en cada renglón las filas con nulos o ceros

data_partido_not_null = data_sin_columnas.dropna(subset = ["Partido"], how = "any")
data_partido_not_empty = data_partido_not_null[data_partido_not_null.Partido != ""]
data_ambientes_not_zero = data_partido_not_empty[(data_partido_not_empty.ambientes_train > 0) | (data_partido_not_empty.ambientes_imputados != 0)]
data_surface_not_zero = data_ambientes_not_zero[data_ambientes_not_zero.surface_covered_in_m2 > 0]
data_surface_not_null = data_surface_not_zero.dropna(subset = ["surface_covered_in_m2"], how = "any")
data_price_not_zero = data_surface_not_null[data_surface_not_null.precio_usd_por_m2 > 0]
data_price_not_null = data_price_not_zero.dropna(subset = ["precio_usd_por_m2"], how = "any")
data_price_not_null

## Outliers

#### Superficie

In [ ]:
q1_surface = data_price_not_null.surface_covered_in_m2.quantile(0.25)
q2_surface = data_price_not_null.surface_covered_in_m2.quantile(0.5)
q3_surface = data_price_not_null.surface_covered_in_m2.quantile(0.75)

higher_bound_surface = q3_surface + 1.5 * (q3_surface - q1_surface)
lower_bound_surface = q1_surface - 1.5 * (q3_surface - q1_surface)

print("El límite inferior es ", lower_bound_surface, " y el superior es ", higher_bound_surface)

# Considerando que el límite inferior da negativo, se usará un estadístico propio para el límite inferior

lower_bound_surface_nuevo = q1_surface.mean() * 0.25
print("El nuevo límite inferior es", lower_bound_surface_nuevo)

outlier_mask_up = data_price_not_null.surface_covered_in_m2 < higher_bound_surface
outlier_mask_down = data_price_not_null.surface_covered_in_m2 > lower_bound_surface_nuevo
outlier_mask = np.logical_and(outlier_mask_up, outlier_mask_down)
data_sin_outliers_superficie = data_price_not_null[outlier_mask]
data_sin_outliers_superficie

#### Precio

In [ ]:
q1_price = data_sin_outliers_superficie.precio_usd_por_m2.quantile(0.25)
q2_price = data_sin_outliers_superficie.precio_usd_por_m2.quantile(0.5)
q3_price = data_sin_outliers_superficie.precio_usd_por_m2.quantile(0.75)

higher_bound_price = q3_price + 1.5 * (q3_price - q1_price)
lower_bound_price = q1_price - 1.5 * (q3_price - q1_price)

print("El límite inferior es ", lower_bound_price, " y el superior es ", higher_bound_price)

# Considerando que el número da negativo, se usará un estadístico propio para el límite inferior

lower_bound_price_nuevo = q1_price.mean() * 0.25
print("El nuevo límite inferior es", lower_bound_price_nuevo)

outlier_mask_up = data_sin_outliers_superficie.precio_usd_por_m2 < higher_bound_price
outlier_mask_down = data_sin_outliers_superficie.precio_usd_por_m2 > lower_bound_price_nuevo
outlier_mask = np.logical_and(outlier_mask_up, outlier_mask_down)
data_sin_outliers_price = data_sin_outliers_superficie[outlier_mask]
data_sin_outliers_price

#### Ambientes

In [ ]:
q1_ambientes = data_sin_outliers_price.ambientes_train.quantile(0.25)
q2_ambientes = data_sin_outliers_price.ambientes_train.quantile(0.5)
q3_ambientes = data_sin_outliers_price.ambientes_train.quantile(0.75)

higher_bound_ambientes = q3_ambientes + 1.5 * (q3_ambientes - q1_ambientes)
lower_bound_ambientes = q1_ambientes - 1.5 * (q3_ambientes - q1_ambientes)
print("El límite inferior es ", lower_bound_ambientes, " y el superior es ", higher_bound_ambientes)

In [ ]:
# Considerando que el límite inferior da negativo, no se usará pues solo tiene lógica que un departamento tenga al menos 1 ambiente.

mask = data_sin_outliers_price.ambientes_train > 7
mask2 = data_sin_outliers_price.loc[mask, :]
data_sin_outliers_ambientes = data_sin_outliers_price.drop(mask2.index, axis = 0)
data_sin_outliers_ambientes

# 5) Exportación del dataset final

In [ ]:
data_final = data_sin_outliers_ambientes.copy()
data_final.to_csv('data_final.csv', index = False, sep=';')